# Puzzle Solver 🧩

In [8]:
# Don't worry about it it just used for easier coding:
import os
import glob

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [9]:
# path of dataset change with your own path
folder_path = r"Puzzle_2_160"

In [10]:
#Read Images
output_file = os.path.join(folder_path, "Output.tif")
patch_files = glob.glob(os.path.join(folder_path, "Patch_*.tif"))

corners_img = np.array(Image.open(output_file))
# Get copy of image to edit it later
base_img = corners_img.copy()

patches = [np.array(Image.open(patch_file)) for patch_file in patch_files]

In [ ]:
plt.imshow(base_img)
plt.axis('off')
plt.show()

In [ ]:
# TODO:
# Now you should solve puzzle do it the best you can.
import numpy as np
import matplotlib.pyplot as plt

def mae(image_ref, image_pred):
    return np.mean(np.abs(image_ref.astype(float) - image_pred.astype(float)))

def puzzle_solver(animate: bool = True, pause_time: float = 0.1):

    from IPython.display import clear_output, display

    # --- Geometry inferred from loaded data ---
    H, W = base_img.shape[:2]
    ph, pw = patches[0].shape[0], patches[0].shape[1]
    rows, cols = H // ph, W // pw

    print(f"Puzzle dimensions: {rows}x{cols}, Piece size: {ph}x{pw}")

    final_img = base_img.copy()
    used_patches = set()  # Track which patches have been used

    # Mark corner pieces as used if they exist
    def mark_corners_as_used():
        corner_coords = [(0, 0), (0, cols-1), (rows-1, 0), (rows-1, cols-1)]
        for (r, c) in corner_coords:
            y0, y1 = r*ph, (r+1)*ph
            x0, x1 = c*pw, (c+1)*pw
            region = final_img[y0:y1, x0:x1, :]

            # Skip if region is empty (black)
            if float(np.mean(region)) < 1.0:
                continue

            # Find matching patch
            for idx, tile in enumerate(patches):
                if idx in used_patches:
                    continue
                if tile.shape == region.shape and np.array_equal(tile, region):
                    used_patches.add(idx)
                    print(f"Corner at ({r},{c}) already placed, marked patch {idx} as used")
                    break

    mark_corners_as_used()

    def is_empty(r, c):
        """Check if cell is empty (black region)."""
        y0, y1 = r*ph, (r+1)*ph
        x0, x1 = c*pw, (c+1)*pw
        return float(np.mean(final_img[y0:y1, x0:x1, :])) < 1.0

    def get_piece_at(r, c):
        """Get the piece currently at position (r, c)."""
        y0, y1 = r*ph, (r+1)*ph
        x0, x1 = c*pw, (c+1)*pw
        return final_img[y0:y1, x0:x1, :]

    def place_piece(r, c, patch_idx):
        """Place a patch at the given position."""
        y0, y1 = r*ph, (r+1)*ph
        x0, x1 = c*pw, (c+1)*pw
        final_img[y0:y1, x0:x1, :] = patches[patch_idx]
        used_patches.add(patch_idx)

    def find_best_patch_for_position(r, c):
        """
        Find the best patch for position (r, c) using multiple neighbor constraints.
        Uses additive scoring when multiple neighbors exist.
        """
        available_patches = [i for i in range(len(patches)) if i not in used_patches]

        if not available_patches:
            return None

        min_cost = float('inf')
        best_patch_idx = None

        for patch_idx in available_patches:
            patch = patches[patch_idx]
            total_cost = 0.0
            neighbor_count = 0

            # Check top neighbor
            if r > 0 and not is_empty(r-1, c):
                top_piece = get_piece_at(r-1, c)
                cost = mae(top_piece[-1:, :, :], patch[0:1, :, :])  # bottom edge of top vs top edge of current
                total_cost += cost
                neighbor_count += 1

            # Check left neighbor
            if c > 0 and not is_empty(r, c-1):
                left_piece = get_piece_at(r, c-1)
                cost = mae(left_piece[:, -1:, :], patch[:, 0:1, :])  # right edge of left vs left edge of current
                total_cost += cost
                neighbor_count += 1

            # Check right neighbor
            if c < cols-1 and not is_empty(r, c+1):
                right_piece = get_piece_at(r, c+1)
                cost = mae(patch[:, -1:, :], right_piece[:, 0:1, :])  # right edge of current vs left edge of right
                total_cost += cost
                neighbor_count += 1

            # Check bottom neighbor
            if r < rows-1 and not is_empty(r+1, c):
                bottom_piece = get_piece_at(r+1, c)
                cost = mae(patch[-1:, :, :], bottom_piece[0:1, :, :])  # bottom edge of current vs top edge of bottom
                total_cost += cost
                neighbor_count += 1

            if neighbor_count > 0:  # Only consider if there are neighbors
                avg_cost = total_cost / neighbor_count  # Average cost per neighbor
                if avg_cost < min_cost:
                    min_cost = avg_cost
                    best_patch_idx = patch_idx

        return best_patch_idx

    # Animation setup
    fig, ax = None, None
    if animate:
        fig, ax = plt.subplots(figsize=(10, 6))
        im = ax.imshow(final_img)
        ax.axis('off')
        ax.set_title("Starting puzzle assembly...")
        clear_output(wait=True)
        display(fig)

    placed_count = len(used_patches)
    total_pieces = rows * cols

    # Phase 1: Fill top row (left to right, skipping corners if already placed)
    print("Phase 1: Filling top row...")
    for c in range(cols):
        if not is_empty(0, c):  # Skip if already filled
            continue

        best_patch = find_best_patch_for_position(0, c)
        if best_patch is not None:
            place_piece(0, c, best_patch)
            placed_count += 1

            if animate:
                im.set_data(final_img)
                ax.set_title(f"Phase 1: Top row - {placed_count}/{total_pieces} pieces placed")
                clear_output(wait=True)
                display(fig)
                time.sleep(pause_time)

    # Phase 2: Fill bottom row (left to right, skipping corners if already placed)
    print("Phase 2: Filling bottom row...")
    for c in range(cols):
        if not is_empty(rows-1, c):  # Skip if already filled
            continue

        best_patch = find_best_patch_for_position(rows-1, c)
        if best_patch is not None:
            place_piece(rows-1, c, best_patch)
            placed_count += 1

            if animate:
                im.set_data(final_img)
                ax.set_title(f"Phase 2: Bottom row - {placed_count}/{total_pieces} pieces placed")
                clear_output(wait=True)
                display(fig)
                time.sleep(pause_time)

    # Phase 3: Fill middle rows (row by row, left to right)
    print("Phase 3: Filling middle rows...")
    for r in range(1, rows-1):  # Skip first and last rows
        for c in range(cols):
            if not is_empty(r, c):  # Skip if already filled
                continue

            best_patch = find_best_patch_for_position(r, c)
            if best_patch is not None:
                place_piece(r, c, best_patch)
                placed_count += 1

                if animate:
                    im.set_data(final_img)
                    ax.set_title(f"Phase 3: Row {r+1} - {placed_count}/{total_pieces} pieces placed")
                    clear_output(wait=True)
                    display(fig)
                    time.sleep(pause_time)

    # Phase 4: Fill any remaining empty spots (cleanup)
    print("Phase 4: Filling any remaining spots...")
    for r in range(rows):
        for c in range(cols):
            if is_empty(r, c):
                best_patch = find_best_patch_for_position(r, c)
                if best_patch is not None:
                    place_piece(r, c, best_patch)
                    placed_count += 1

                    if animate:
                        im.set_data(final_img)
                        ax.set_title(f"Phase 4: Cleanup - {placed_count}/{total_pieces} pieces placed")
                        clear_output(wait=True)
                        display(fig)
                        time.sleep(pause_time)

    if animate:
        ax.set_title(f"Puzzle Complete! {placed_count}/{total_pieces} pieces placed")
        clear_output(wait=True)
        display(fig)

    print(f"Puzzle assembly complete! Placed {placed_count}/{total_pieces} pieces")
    return final_img


In [ ]:
# Test and Save final Image
final_img = puzzle_solver()
Image.fromarray(final_img).save(os.path.join(folder_path, "final.tif"))

In [ ]:
# Now it's time to evaluate how good the final image is versus the original image
original_img = np.array(Image.open(os.path.join(folder_path, "Original.tif")))

# Auto-detect grid size from image and patch sizes (no hardcoding)
ph, pw = patches[0].shape[0], patches[0].shape[1]
rows = original_img.shape[0] // ph
cols = original_img.shape[1] // pw
patch_height, patch_width = ph, pw

def accuracy_block_base(original_img, final_img, rows, cols):
    correct_blocks = 0
    total_blocks = rows * cols
    for row in range(rows):
        for col in range(cols):
            patch = final_img[row*patch_height:(row+1)*patch_height, col*patch_width:(col+1)*patch_width]
            original_patch = original_img[row*patch_height:(row+1)*patch_height, col*patch_width:(col+1)*patch_width]
            if np.array_equal(patch, original_patch):
                correct_blocks += 1
    return 100 * correct_blocks / total_blocks

accuracy = accuracy_block_base(original_img, final_img, rows, cols)
print(f"Accuracy: {accuracy:.2f}%  (rows={rows}, cols={cols}, patch={patch_height}x{patch_width})")

In [ ]:
# show final_img - You can make animation for it while solving puzzle. That's fun!
plt.imshow(final_img)
plt.axis('off')
plt.show()